In [10]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import numpy as np
import bocd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import yfinance as yf

from machine_learning_finance import (analyze_trades, 
    calc_probabilties_without_lookahead, plot_backtest_analysis, 
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for,
    calculate_polynomial_regression, TraderEnv, make_price_marker_from_boolean, add_marker, add_line)

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "MSFT"
file = f"../backtests/{symbol}-model-back-test.csv"
period = 365
pd.options.display.max_rows = None
inverse = None # set to anything to graph inverse longs

def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()


        
def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)    
    test_df = calc_probabilties_without_lookahead(test_df, hist_df, hist_cutoff="2023-05-15")
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, None, 365 * 4, None, 365)
    env = TraderEnv(symbol, test_df, hist_df)
    
    # Todo here: we have all the data we need to render our graphs in the env already
    # We need to modify plot_backtest_analysis to:
    # * render change points
    # * render new moving averages
    # * remove all the probability stuff
    # * Figure out how to supply the polynomials!!!
    
    fig = plot_backtest_analysis(env.orig_timeseries_with_analytics, ledger, inverse=inverse)

    # convert each change point column to price of the average in question
    analytics_df = env.orig_timeseries_with_analytics
    periods=[30, 60, 90]
    colors=["blue", "purple", "green"]
    for period, color in zip(periods, colors):        
        marker_col =  f"cp-loc-{period}"
        trend_col = f"trend-{period}"
        analytics_df = make_price_marker_from_boolean(analytics_df, f"change-point-{period}", f"trend-{period}", marker_col) 
        add_marker(fig, analytics_df, marker_col, f"change-point-{period}", "diamond", 8, color)
        add_line(fig, analytics_df, trend_col, f"moving-avg-{period}", color)
    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)
    fig.show()

    
    fig = make_subplots(rows=3, cols=1)
    indexes=[1,2,3]
    for period, color, index in zip(periods, colors, indexes):        
        fig.add_trace(go.Bar(x=analytics_df.index, y=analytics_df[f'polynomial_derivative-{period}'], marker=dict(color=f'{color}', colorscale='Viridis'), name=f'Polynomial Derivative {period}'), row=index, col=1)

    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)
    fig.show()

    return ledger, test_df, env

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)


ledger, df2, env = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 
env.orig_timeseries_with_analytics
#df2[["moving_avg", "change_points", "rt", "Close"]]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,trend-30,trend-diff-30,change-point-30,polynomial_derivative-30,...,trend-diff-90,change-point-90,polynomial_derivative-90,long_entry,long_exit,short_entry,short_exit,cp-loc-30,cp-loc-60,cp-loc-90
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-07,306.170013,307.839996,299.899994,300.950012,296.762573,28533300,313.549668,-12.599656,False,-0.539094,...,-18.345211,False,0.180225,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-08,301.250000,305.559998,299.950012,304.559998,300.322357,32421200,312.286668,-7.726670,False,-0.530125,...,-14.986781,False,0.155398,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-09,309.869995,311.929993,307.390015,311.209991,306.879791,31284700,311.285334,-0.075343,False,-0.521155,...,-8.582453,False,0.130571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-10,304.040009,309.119995,300.700012,302.380005,298.172638,45386200,309.966334,-7.586329,False,-0.512186,...,-17.626551,False,0.105744,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-11,303.190002,304.290009,294.220001,295.040009,290.934784,39175600,308.490334,-13.450326,False,-0.503216,...,-25.036325,False,0.080917,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-14,293.769989,296.760010,291.350006,295.000000,290.895325,36359500,307.113000,-12.113000,False,-0.494247,...,-25.097334,False,0.056090,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-15,300.010010,300.799988,297.019989,300.470001,296.289276,27058300,305.970334,-5.500333,False,-0.485277,...,-19.689777,False,0.031264,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-16,298.369995,300.869995,293.679993,299.500000,295.943390,29982100,304.986667,-5.486667,False,-0.476307,...,-20.711445,False,0.006437,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-17,296.359985,296.799988,290.000000,290.730011,287.277557,32461600,304.131667,-13.401656,False,-0.467338,...,-29.442545,False,-0.018390,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ledger